In [344]:
import os
%pwd

'C:\\Users\\91638\\OneDrive\\Documents\\Chicken-Disease-Classification-Project\\Chicken-Disease-Classification-Project'

In [345]:
new_directory = r"C:\Users\91638\OneDrive\Documents\Chicken-Disease-Classification-Project\Chicken-Disease-Classification-Project"  # Replace this with the actual path

# Change the working directory
os.chdir(new_directory)

In [346]:
%pwd

'C:\\Users\\91638\\OneDrive\\Documents\\Chicken-Disease-Classification-Project\\Chicken-Disease-Classification-Project'

In [347]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [348]:
from src.cnn_Classifier.constants import *
from src.cnn_Classifier.utils.common import read_yaml, create_directories

In [349]:
# Get current working directory
current_working_directory = Path(os.getcwd())
print(f"Current Working Directory: {current_working_directory}")

Current Working Directory: C:\Users\91638\OneDrive\Documents\Chicken-Disease-Classification-Project\Chicken-Disease-Classification-Project


In [350]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params= read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
             root_dir=config.root_dir,
             source_URL=config.source_URL,
             local_data_file=config.local_data_file, 
             unzip_dir=config.unzip_dir)

        return data_ingestion_config    

In [351]:
import os

import urllib.request as request

import zipfile

from src.cnn_Classifier import logger

from src.cnn_Classifier.utils.common import get_size

In [352]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
         filename,headers=  request.urlretrieve(
            url= self.config.source_URL,
            filename= self.config.local_data_file
         )
         logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
           logger.info(f"File already exists of Size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Fun tion returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [353]:
from src.cnn_Classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.cnn_Classifier.entity.config_entity import DataIngestionConfig

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Debug print to check if the config is loaded correctly
        print("Loaded Config:", self.config)

        create_directories([self.config['artifacts_root']])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']

        # Debug print to check if the data_ingestion config is loaded correctly
        print("Data Ingestion Config:", config)

        create_directories([config['root_dir']])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_URL=config['source_URL'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )

        return data_ingestion_config

In [354]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'Chicken-Disease-Classification-Project\\config\\config.yaml'

In [363]:
import requests

url= "https://github.com/Shakthi-2004/task_manager2/raw/main/Chicken-fecal-images.zip"
response = requests.get(url)

print("Status Code:", response.status_code)
if response.status_code == 200:
    print("Content-Type:", response.headers['Content-Type'])
    with open("test_download.zip", 'wb') as f:
        f.write(response.content)
else:
    print("Failed to retrieve the file.")


Status Code: 404
Failed to retrieve the file.


In [365]:
import os
import requests

url = "https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip"
response = requests.get(url)

print("Status Code:", response.status_code)
if response.status_code == 200:
    print("Content-Type:", response.headers['Content-Type'])
    with open("test_download.zip", 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to retrieve the file.")


Status Code: 200
Content-Type: application/zip
File downloaded successfully.


In [361]:
import logging
from cnn_Classifier.config.configuration import ConfigurationManager
from cnn_Classifier.pipeline.stage_01_data_ingestion import DataIngestion

# Setting up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

if __name__ == '__main__':
    try:
        STAGE_NAME = "Data Ingestion Stage"
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")

        config_manager = ConfigurationManager()
        data_ingestion_config = config_manager.get_data_ingestion_config()

        data_ingestion = DataIngestion(config=data_ingestion_config)
        
        # Print statements for debugging
        print("Data Ingestion Config Root Dir:", data_ingestion_config.root_dir)
        print("Data Ingestion Config Source URL:", data_ingestion_config.source_URL)
        print("Data Ingestion Config Local Data File:", data_ingestion_config.local_data_file)
        print("Data Ingestion Config Unzip Dir:", data_ingestion_config.unzip_dir)
        
        data_ingestion.download_file()

        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<<\n\nx==========x")

    except Exception as e:
        logger.exception(e)
        raise e


[2024-06-17 14:27:34,621: INFO: 1049045141: >>>>>> stage Data Ingestion Stage started <<<<<<]
[2024-06-17 14:27:34,628: INFO: common: yaml file: config\config.yaml loaded successfully!]
[2024-06-17 14:27:34,640: INFO: common: yaml file: params.yaml loaded successfully!]
[2024-06-17 14:27:34,643: INFO: common: created directory at: artifacts]
[2024-06-17 14:27:34,645: INFO: common: created directory at: artifacts/data_ingestion]


Data Ingestion Config Root Dir: artifacts/data_ingestion
Data Ingestion Config Source URL: https://github.com/Shakthi-2004/task_manager2/raw/main/Chicken-fecal-images.zip
Data Ingestion Config Local Data File: artifacts/data_ingestion/data.zip
Data Ingestion Config Unzip Dir: artifacts/data_ingestion
[2024-06-17 14:27:35,019: ERROR: 1049045141: HTTP Error 404: Not Found]
Traceback (most recent call last):
  File "C:\Users\91638\AppData\Local\Temp\ipykernel_47020\1049045141.py", line 25, in <module>
    data_ingestion.download_file()
  File "c:\users\91638\onedrive\documents\chicken-disease-classification-project\chicken-disease-classification-project\src\cnn_Classifier\components\data_ingestion.py", line 20, in download_file
    filename,headers=  request.urlretrieve(
  File "c:\Users\91638\anaconda3\lib\urllib\request.py", line 239, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "c:\Users\91638\anaconda3\lib\urllib\request.py", line 214, in urlopen
    re

HTTPError: HTTP Error 404: Not Found